In [15]:
from datasets import Dataset, DatasetDict
from transformers import BertModel, BertTokenizer, BertConfig, Trainer, TrainingArguments, AutoModelForSequenceClassification
import pandas as pd
import evaluate
import numpy as np
from optuna.trial import Trial

In [2]:
df = pd.read_csv('./data/processed_df.csv', index_col=0)
df = df[['HER2', 'ER', 'PR', 'HR', 'text', 'SUBTYPE']]
augmented_train_df = pd.read_csv('./augmented_train_df3.csv', index_col=0)
test_df = df[~df.index.isin(augmented_train_df.index)]

In [3]:
id2label = {str(i): label
           for i, label in enumerate(df['SUBTYPE'].unique())}
label2id = {v: k for k, v in enumerate(df['SUBTYPE'].unique())}

# augmented_train_df['label'] = augmented_train_df['SUBTYPE'].apply(lambda x: label2id[x])
# test_df['label'] = test_df['SUBTYPE'].apply(lambda x: label2id[x])

In [4]:
train_ds = Dataset.from_pandas(augmented_train_df)
test_ds = Dataset.from_pandas(test_df)

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_config = BertConfig(num_hidden_layers=8, num_attention_heads=8)
bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# # Defining audio maximum duration
# max_duration = 30.0 # 30 seconds

# def preprocess_function(text_ds: Dataset):
#     texts = text_ds['text']
#     # print(texts)

#     return bert_tokenizer.encode_plus(
#             texts,
#             add_special_tokens=True,
#             padding='max_length',
#             return_token_type_ids=True,
#             truncation=True
#         )

# mapped_train_ds = train_ds.map(preprocess_function, remove_columns=['HER2', 'ER', 'PR', 'HR', 'text', 'SUBTYPE', '__index_level_0__'])
# mapped_test_ds = test_ds.map(preprocess_function, remove_columns=['HER2', 'ER', 'PR', 'HR', 'text', 'SUBTYPE', '__index_level_0__'])

# ds = DatasetDict()
# ds['train'] = mapped_train_ds
# ds['test'] = mapped_test_ds

# ds.save_to_disk('tcga-dataset')

NameError: name 'train_ds' is not defined

In [5]:
ds = DatasetDict.load_from_disk('tcga-dataset')

In [6]:
training_args = TrainingArguments(
    output_dir = 'bert_output',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
    learning_rate = 5e-5,
    seed = 42,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 1,
    num_train_epochs = 15,
    warmup_ratio = 0.1,
    fp16 = True,
    save_total_limit = 2,
    report_to = 'none'
    )

In [7]:
metric = evaluate.load('accuracy')
# Creating function to compute accuracy
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis = 1)
    return metric.compute(predictions = predictions, references = eval_pred.label_ids)

In [16]:
def optuna_hp_space(trial: Trial):
    trial.set_user_attr()
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "dropout_rate": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
    }

In [26]:
def model_init(trial: Trial=None):
    if trial:
        
        print(trial.user_attrs, trial.params, trial.relative_params, trial.system_attrs)
    bert_config = BertConfig(num_hidden_layers=8, num_attention_heads=8)
    bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)
    return bert_model

In [27]:
trainer = Trainer(
    model_init=model_init,
    args = training_args,
    train_dataset = ds['train'],
    eval_dataset = ds['test'],
    # hp_space=optuna_hp_space,
    compute_metrics = compute_metrics)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
best_trials = trainer.hyperparameter_search(
    direction=["minimize", "maximize", "minimize"],
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=20,
    compute_objective=compute_metrics,
)

[I 2024-02-25 23:56:46,397] A new study created in memory with name: no-name-0d60a9a1-9c3d-4096-857b-19c2ff2520d9
Trying to set dropout_rate in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
C:\Users\jswietek\AppData\Local\Temp\3\ipykernel_3708\2487656704.py:4: FutureWarning: system_attrs has been deprecated in v3.1.0. This feature will be removed in v5.0.0. See https://github.com/optuna/optuna/releases/tag/v3.1.0.
  print(trial.user_attrs, trial.params, trial.relative_params, trial.system_attrs)


{} {'learning_rate': 2.679042240108809e-06, 'per_device_train_batch_size': 16} {} {'nsga2:generation': 0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[W 2024-02-25 23:56:48,314] Trial 0 failed with parameters: {'learning_rate': 2.679042240108809e-06, 'per_device_train_batch_size': 16} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacty of 12.00 GiB of which 0 bytes is free. Of the allocated memory 9.58 GiB is allocated by PyTorch, and 135.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "d:\jswietek\mgr\env\lib\site-packages\optuna\study\_optimize.py", line 200, in _

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacty of 12.00 GiB of which 0 bytes is free. Of the allocated memory 9.58 GiB is allocated by PyTorch, and 135.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [12]:
trainer.train()

                                                    
  7%|▋         | 358/5370 [05:33<1:10:53,  1.18it/s]

{'eval_loss': 1.3671650886535645, 'eval_accuracy': 0.3333333333333333, 'eval_runtime': 4.3606, 'eval_samples_per_second': 13.76, 'eval_steps_per_second': 1.835, 'epoch': 1.0}


  9%|▉         | 500/5370 [08:13<1:13:48,  1.10it/s] 

{'loss': 0.9261, 'learning_rate': 4.646182495344507e-05, 'epoch': 1.4}


                                                    
 13%|█▎        | 716/5370 [11:30<1:05:20,  1.19it/s]

...

                                                     
100%|██████████| 5370/5370 [1:27:35<00:00,  1.19it/s]

{'eval_loss': 5.458714962005615, 'eval_accuracy': 0.4166666666666667, 'eval_runtime': 1.3494, 'eval_samples_per_second': 44.466, 'eval_steps_per_second': 5.929, 'epoch': 15.0}


100%|██████████| 5370/5370 [1:28:01<00:00,  1.02it/s]

{'train_runtime': 5281.2633, 'train_samples_per_second': 16.257, 'train_steps_per_second': 1.017, 'train_loss': 0.22136466303351204, 'epoch': 15.0}


TrainOutput(global_step=5370, training_loss=0.22136466303351204, metrics={'train_runtime': 5281.2633, 'train_samples_per_second': 16.257, 'train_steps_per_second': 1.017, 'train_loss': 0.22136466303351204, 'epoch': 15.0})